# Paralelization for gradient-free optimizers: Differential Evolution
As in the rest of the examples, all the imports are done and the vQPUs are raised. After this, they are brought to the program workflow in form of `QPU` instances.

In [ ]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from scipy.stats import entropy, norm
from scipy.optimize import differential_evolution
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter

sys.path.append(os.getenv("HOME"))

from cunqa import get_QPUs, qraise, qdrop
from cunqa.qpu import run 
from cunqa.mappers import QJobMapper, QPUCircuitMapper

family = qraise(10, "00:20:00", simulator = "Aer",  co_located = True)
qpus  = get_QPUs(co_located = True, family = family)

for q in qpus:
    print(f"QPU {q.id}, backend: {q.backend.name}, simulator: {q.backend.simulator}, version: {q.backend.version}.")

In this example, the same ansatz, target distribution and distribution divergence measure is defined as in the parameters update feature example. 

In [ ]:
def hardware_efficient_ansatz(num_qubits, num_layers):
    qc = QuantumCircuit(num_qubits)
    param_idx = 0
    for _ in range(num_layers):
        for qubit in range(num_qubits):
            phi = Parameter(f'phi_{param_idx}_{qubit}')
            lam = Parameter(f'lam_{param_idx}_{qubit}')
            qc.ry(phi, qubit)
            qc.rz(lam, qubit)
        param_idx += 1
        for qubit in range(num_qubits - 1):
            qc.cx(qubit, qubit + 1)
    qc.measure_all()
    return qc

def target_distribution(num_qubits):
    # Define a normal distribution over the states
    num_states = 2 ** num_qubits
    states = np.arange(num_states)
    mean = num_states / 2
    std_dev = num_states / 4
    target_probs = norm.pdf(states, mean, std_dev)
    target_probs /= target_probs.sum()  # Normalize to make it a valid probability distribution
    target_dist = {format(i, f'0{num_qubits}b'): target_probs[i] for i in range(num_states)}
    return target_dist

def KL_divergence(counts, n_shots, target_dist):
    # Convert counts to probabilities
    pdf = pd.DataFrame.from_dict(counts, orient="index").reset_index()
    pdf.rename(columns={"index": "state", 0: "counts"}, inplace=True)
    pdf["probability"] = pdf["counts"] / n_shots
    
    # Create a dictionary for the obtained distribution
    obtained_dist = pdf.set_index("state")["probability"].to_dict()
    
    # Ensure all states are present in the obtained distribution
    for state in target_dist:
        if state not in obtained_dist:
            obtained_dist[state] = 0.0
    
    # Convert distributions to lists for KL divergence calculation
    target_probs = [target_dist[state] for state in sorted(target_dist)]
    obtained_probs = [obtained_dist[state] for state in sorted(obtained_dist)]
    
    # Calculate KL divergence
    kl_divergence = entropy(obtained_probs, target_probs)
    
    return kl_divergence

num_qubits = 6
num_layers = 3
ansatz = hardware_efficient_ansatz(num_qubits, num_layers)

pop=[np.random.uniform(-np.pi, np.pi, ansatz.num_parameters) for _ in range(ansatz.num_parameters)]
bounds=[(-np.pi,np.pi) for _ in range(ansatz.num_parameters)]

What changes is the cost function, that in this case will be the following.

In [ ]:
def cost_function(result):
    target_dist = target_distribution(num_qubits)
    counts = result.counts
    
    return KL_divergence(counts, 1000, target_dist)

In [ ]:
def make_callback(mapper):
    i = 0
    pbar = tqdm(desc="Optimization", unit="iter")
    
    best_individual = []
    energies = []
    def callback(xk, convergence = 1e-8):
        nonlocal i
        best_individual.append(xk)
        energy = mapper(cost_function, [xk])[0]
        energies.append(energy)
        
        pbar.update(1)
        pbar.set_postfix(fx=f"{energy:.3e}")
        i += 1

    def close():
        pbar.close()

    return callback, close, energies

### QJobMapper

In [ ]:
init_qjobs = []
for i in range(1*ansatz.num_parameters):# we set pop=1 as the population size is pop*ansatz.num_parameters
    qpu = qpus[i%len(qpus)]# we select the qpu
    init_qjobs.append(run(ansatz.assign_parameters(np.zeros(ansatz.num_parameters)), qpu, shots=1000))

mapper = QJobMapper(init_qjobs)

In [ ]:
callback1, close1, energies1 = make_callback(mapper)
result1 = differential_evolution(cost_function, 
                                bounds, 
                                maxiter = 1000, 
                                workers = mapper, 
                                updating = 'deferred',
                                strategy = 'best1bin', 
                                init = pop, 
                                polish = False, 
                                callback=callback1
                               )
close1()

In [ ]:
print(result1)

### QPUCircuitMapper

In [ ]:
mapper = QPUCircuitMapper(qpus, ansatz, transpile=False, shots=1000)

In [ ]:
callback2, close2, energies2 = make_callback(mapper)
result2 = differential_evolution(cost_function, 
                                bounds, 
                                maxiter = 1000, 
                                workers = mapper, 
                                updating = 'deferred',
                                strategy = 'best1bin', 
                                init = pop, 
                                polish = False, 
                                callback=callback2
                               )
close2()

In [ ]:
%matplotlib inline
plt.clf()
plt.plot(np.linspace(0, result1.nit, result1.nit), energies1, label="Optimization path (QJobMapper)")
upper_bound = result1.nit
plt.plot(np.linspace(0, result2.nit, result2.nit), energies2, label="Optimization path (QPUCircuitMapper)")
plt.plot(np.linspace(0, upper_bound, upper_bound), np.zeros(upper_bound), "--", label="Target cost")
plt.xlabel("Step"); plt.ylabel("Cost");
plt.legend(loc="upper right");
plt.title(f"n = {num_qubits}, l = {num_layers}, # params = {ansatz.num_parameters}")
plt.grid(True)
plt.show()

# Paralelization of expectation value terms

In [ ]:
# TODO

# Paralelization for gradient optimizers

In [ ]:
# TODO